In [1]:
import degirum as dg, degirum_tools
import os
from pathlib import Path
import gen_test_image_util as imgutil
#import importlib
import os
import time
import cv2
#importlib.reload(imgutil)

In [2]:
hw_location = "@cloud"
#hw_location = "@local"
#model_zoo_url = "https://cs.degirum.com/degirum/public"
model_zoo_url = "https://cs.degirum.com/degirum/ultralytics_v6"
#device_family = "ORCA1"  # models of which device family to use
device_family = "EDGETPU"
image_dir = Path('/home/gotom/data_dir/split_pic')

In [3]:
#create image chunk 
image_sizeL = [224, 224, 200]
image_chunkL = imgutil.create_test_image_chunk('maami_1.jpg', image_sizeL)

size:224x224
726


In [4]:
#save image chunk
image_chunks_pathL = imgutil.save_image_chunk(image_dir, image_chunkL)

In [5]:
def create_model_instances(model_name, zoo, batch_size, use_jpeg):

    model_instance = zoo.load_model(model_name)
    model_instance.image_backend = "pil"  # select OpenCV backend
    model_instance.input_numpy_colorspace = "RGB"      
    model_instance._model_parameters.InputImgFmt = ["JPEG" if use_jpeg else "RAW"]
    model_instance.measure_time = True
    model_instance.eager_batch_size = batch_size
    model_instance.frame_queue_depth = batch_size

    return model_instance

In [6]:
def print_results(target_modelsL, batch_size, image_sizeL, durationL, observed_fpsL, expected_fpsL):
    
    CW = (65,30, 16, 16)  # column widths
    header = f"| {'Model name':{CW[0]}}| {f'{image_sizeL[2]} Images Inference Time[ms]':{CW[1]}} | {'Observed FPS':{CW[2]}} | {'Max Possible FPS':{CW[3]}} |"

    print(f"hw_location:  {hw_location}")
    print(f"model_zoo_url:{model_zoo_url}")
    print(f'Batch Size:   {batch_size}')
    print(f'Image Size:   {image_sizeL[0]} x {image_sizeL[1]}')
    print(f"{'-'*len(header)}")
    print(header)
    print(f"{'-'*len(header)}")

    for mi, model in enumerate(target_modelsL):
        print(f"| {target_modelsL[mi]:{CW[0]}}|" + f" {durationL[mi]:{CW[1]}} |" + f" {observed_fpsL[mi]:{CW[2]}} |" + f" {expected_fpsL[mi]:{CW[3]}} |")
    
    print(f"{'-'*len(header)}")    

In [7]:
# save results
def save_results(target_modelsL, batch_size, image_sizeL, durationL, observed_fpsL, expected_fpsL):
    CW = (65,30, 16, 16)  # column widths
    header = f"| {'Model name':{CW[0]}}| {f'{image_sizeL[2]} Images Inference Time[ms]':{CW[1]}} | {'Observed FPS':{CW[2]}} | {'Max Possible FPS':{CW[3]}} |"

    d = time.strftime('%m%d%H%M')
    data_dir = '/home/gotom/data_dir/200pics_measure/' 
    file = data_dir + 'test_results' + d + '.txt'

    f = open(file, 'w')
    f.write(f"hw_location:  {hw_location}\n")
    f.write(f"model_zoo_url:{model_zoo_url}\n")
    f.write(f'Batch Size:   {batch_size}\n')
    f.write(f'Image Size:   {image_sizeL[0]} x {image_sizeL[1]}\n')
    f.write(f"{'-'*len(header)}\n")
    f.write(header + '\n')
    f.write(f"{'-'*len(header)}\n")

    for mi, model in enumerate(target_modelsL):
    
        f.write(f"| {target_modelsL[mi]:{CW[0]}}|" + f" {durationL[mi]:{CW[1]}} |" + f" {observed_fpsL[mi]:{CW[2]}} |" + f" {expected_fpsL[mi]:{CW[3]}} |\n")
    
    f.write(f"{'-'*len(header)}\n")
    f.close()

In [8]:
zoo = dg.connect(hw_location, model_zoo_url, degirum_tools.get_token()) # connect to AI inference engine
model_name = zoo.list_models(device=device_family)

#target_modelsL = ['mobilenet_v1_imagenet--224x224_quant_n2x_orca1_1', 'mobilenet_v2_imagenet--224x224_quant_n2x_orca1_1']

target_modelsL = []

#target_modelsL = model_name

for name in model_name:

    if not name.find('coco') == -1:
        #print(f'mame: {name}')
        target_modelsL.append(name)
               
for name in target_modelsL:
    
    if name.find('sound') > 0:
        #print(f'mame: {name}')
        target_modelsL.remove(name)
        
    elif name.find('silu') > 0:
        print(f'mame: {name}')
        target_modelsL.remove(name)
        

mame: yolov5nu_silu_coco--640x640_quant_tflite_edgetpu_1
mame: yolov5su_silu_coco--640x640_quant_tflite_edgetpu_1
mame: yolov8n_silu_coco--640x640_quant_tflite_edgetpu_1
mame: yolov8s_silu_coco--640x640_quant_tflite_edgetpu_1


In [9]:
#get image chunk
image_dir = Path('/home/gotom/data_dir/split_pic/')
image_chunk_pathL = imgutil.get_image_chunk_path(image_dir, image_sizeL[2])
print(image_chunk_pathL[image_sizeL[2] -1])

/home/gotom/data_dir/split_pic/chunk_199.jpg


In [10]:
exclude_preprocessing = True
use_jpeg = True

batch_sizeL = [2, 4, 8, 16]

durationL = []
observed_fpsL = []
expected_fpsL = []

batch_size = batch_sizeL[3]
for mi, model_name in enumerate(target_modelsL):
    print(f'model_name: {model_name}')
    model_instance = create_model_instances(model_name, zoo, batch_size, use_jpeg) 
    
    image = cv2.imread(image_chunk_pathL[0], cv2.IMREAD_COLOR)
    h,w = image.shape[:2]
    image_sizeL[0] = h
    image_sizeL[1] = w
    
    print(f'image_size; {image_sizeL[0]} x {image_sizeL[1]}')
    #print(str(image_chunks_pathL[0]))  
    model_instance(image_chunk_pathL[0]) #dummy

    model_instance.reset_time_stats()

    count = 0
    t = degirum_tools.Timer()
    
    for res in model_instance.predict_batch(imgutil.pics_transaction_gen(model_instance, image_chunk_pathL, exclude_preprocessing, image_sizeL[2])):
        count += 1
        #pass

    print(f'count = {count}')
    
    if not(count == image_sizeL[2]): 
       exit()
     
    elapsed_time = round(1e3*t(),1)   
    durationL.append(elapsed_time)
    
    time_stats = model_instance.time_stats()
    inference_ms = time_stats["CoreInferenceDuration_ms"].avg
    frame_duration_ms = elapsed_time / image_sizeL[2]
    observed_fps_value = round(1e3 / frame_duration_ms, 1) 
    expected_fps_value = round(1e3 / inference_ms, 1)
    
    observed_fpsL.append(observed_fps_value)
    expected_fpsL.append(expected_fps_value)
    
    

model_name: yolov5nu_relu6_coco--640x640_quant_tflite_edgetpu_1
image_size; 224 x 224
count = 200
model_name: yolov5su_relu6_coco--640x640_quant_tflite_edgetpu_1
image_size; 224 x 224
count = 200
model_name: yolov8n_relu6_coco--640x640_quant_tflite_edgetpu_1
image_size; 224 x 224
count = 200
model_name: yolov8s_relu6_coco--640x640_quant_tflite_edgetpu_1
image_size; 224 x 224
count = 200


In [11]:
print_results(target_modelsL, batch_size, image_sizeL, durationL, observed_fpsL, expected_fpsL)

hw_location:  @cloud
model_zoo_url:https://cs.degirum.com/degirum/ultralytics_v6
Batch Size:   16
Image Size:   224 x 224
-------------------------------------------------------------------------------------------------------------------------------------------
| Model name                                                       | 200 Images Inference Time[ms]  | Observed FPS     | Max Possible FPS |
-------------------------------------------------------------------------------------------------------------------------------------------
| yolov5nu_relu6_coco--640x640_quant_tflite_edgetpu_1              |                       476997.5 |              0.4 |              0.4 |
| yolov5su_relu6_coco--640x640_quant_tflite_edgetpu_1              |                      1298633.7 |              0.2 |              0.2 |
| yolov8n_relu6_coco--640x640_quant_tflite_edgetpu_1               |                       542933.4 |              0.4 |              0.4 |
| yolov8s_relu6_coco--640x640_quant_tf

In [13]:
save_results(target_modelsL, batch_size, image_sizeL, durationL, observed_fpsL, expected_fpsL)